In [1]:
import os 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, log_loss 
import pickle
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
base_dir_mit = '/data/ddmg/scate/multiple_shortcut/dr'
base_dir_gl = '/nfs/turbo/coe-soto/mmakar/multiple_shortcut/dr'
if os.path.exists(base_dir_mit):
    base_dir = base_dir_mit
elif os.path.exists(base_dir_gl):
    base_dir = base_dir_gl 
pixel = 299
batch_size = 64

In [3]:
def get_weighted_micro_auc(df):
    df = pd.concat([df, pd.get_dummies(df.y0, prefix='y0')], axis=1)
    final_df = []
    for y_val in df.y0.unique().tolist():
        curr_df = df[[f'y0_{y_val}',  f'predictions{int(y_val)}', 'sample_weights']]
        curr_df.rename(columns={f'y0_{y_val}': 'y_true', 
            f'predictions{int(y_val)}': 'y_pred'}, inplace=True)
        final_df.append(curr_df)
    
    final_df = pd.concat(final_df, ignore_index=True, axis=0)
    return roc_auc_score(final_df.y_true, final_df.y_pred, sample_weight=final_df.sample_weights)

In [4]:
def single_class_auc(y_cat_true, y_cat_pred, single_class, sample_weights):
    y_true = (y_cat_true == single_class) * 1
    y_pred = y_cat_pred[[f'predictions{single_class}']]
    if sample_weights is None:
        return roc_auc_score(y_true, y_pred)
    return roc_auc_score(y_true, y_pred, sample_weight=sample_weights)

def get_per_dist_metrics(df, model_name):
    model_eval = {'hash': df.model.iloc[0]}
    if 'unweighted' in model_name:
        model_eval['validation_macro_auc'] = roc_auc_score(df.y0, df[
            [f'predictions{i}' for i in range(5)]], 
            average='macro', multi_class='ovr')
        model_eval['validation_micro_auc'] = roc_auc_score(df.y0, df[
            [f'predictions{i}' for i in range(5)]], 
            average='macro', multi_class='ovo')
        model_eval['validation_pred_log_loss'] = log_loss(df.y0, df[
            [f'predictions{i}' for i in range(5)]])    
    else: 
        # --- getting macro auc
        all_aucs = []
        for single_class in range(5):
            curr_class_auc = single_class_auc(df.y0, df[
                [f'predictions{i}' for i in range(5)]], single_class,
                                              sample_weights=df.sample_weights)
            all_aucs.append(curr_class_auc)
            
        model_eval[f'validation_macro_auc'] = np.mean(all_aucs)
        # --- getting micro auc 
        model_eval[f'validation_micro_auc'] = get_weighted_micro_auc(df)
        
        #--- getting log loss 
        model_eval['validation_pred_log_loss'] = log_loss(df.y0, df[
            [f'predictions{i}' for i in range(5)]], sample_weight=df.sample_weights)
        
    return model_eval 

    
def update_performance(df, model_name): 
    old_eval = pickle.load(open(
        f"{base_dir}/tuning/{df['model'].iloc[0]}/performance.pkl", "rb"))
#     if 'validation_micro_auc' in old_eval.keys():
#         return old_eval 
    model_eval = get_per_dist_metrics(df, model_name)
    old_eval.update(model_eval)
    pickle.dump(old_eval, open(f"{base_dir}/tuning/{model_eval['hash']}/performance.pkl", 
                    'wb'))
    return model_eval
    

# def get_all_dist_metrics(random_seeds, model_name, xv_mode, py1y0=0.9):
#     all_eval = []
#     for random_seed in random_seeds: 
#         curr_pred = pd.read_csv(
#             (f'{base_dir}/final_models/opt_pred_rs{random_seed}_{model_name}_{xv_mode}'
#              f'_pix{pixel}_bs{batch_size}_py1y0{py1y0}_epochs2.csv')) 
#         all_eval.append(get_per_dist_metrics(curr_pred))
        
#     all_eval = pd.concat(all_eval, ignore_index=True)
#     if len(random_seeds) > 1:
#         all_eval = all_eval.groupby('dist').agg(['mean', 'std']).reset_index()
#         all_eval.columns = ['_'.join(col).strip() for col in all_eval.columns.values]
#         all_eval.rename(columns={'dist_': 'dist'}, inplace=True)
#     return all_eval


def get_all_dist_metrics_all_models(model_name, xv_mode, base_dir,  py1y0=0.9):
    all_eval = []
    curr_pred = pd.read_csv(
        (f'{base_dir}/final_models/all_valid_pred_{model_name}_{xv_mode}'
         f'_pix{pixel}_bs{batch_size}_py1y0{py1y0}_epochs10.csv')) 
    for model in curr_pred.model.unique().tolist():
        model_pred = curr_pred[(curr_pred.model == model)]
        model_eval = update_performance(model_pred, model_name)
        all_eval.append(pd.DataFrame(model_eval, index=[0]))
    all_eval = pd.concat(all_eval, ignore_index=True)
    return all_eval
            

In [5]:
model_name = 'weighted_hsic'
xv_mode = 'two_step'
valid_eval = get_all_dist_metrics_all_models(model_name, xv_mode, base_dir=base_dir)

In [6]:
valid_eval.shape

(12, 4)

In [28]:
dist = 0.1
experiment_directory = (
    f"{base_dir}/experiment_data/rs0")

test_data = pd.read_csv(
    f'{experiment_directory}/test_{dist}.txt'
)
test_data = test_data['0'].str.split(",", expand=True)
D = test_data.shape[1] - 2
test_data.columns = ['img_name', 'noise_img'] + [f'y{i}' for i in range(D)]
test_data['group'] = test_data.y0.map(str) + "-" + test_data.y1.map(str)
test_data = test_data.groupby('group').sample(frac=0.2, random_state=0).reset_index(drop=True)

txt_data = test_data.img_name + ',' + test_data.noise_img

for i in range(D):
    txt_data = txt_data + ',' + test_data[f'y{i}'].astype(str)
txt_data = txt_data.apply(lambda x: [x])
txt_data = txt_data.values.tolist()
txt_data = [
    tuple(txt_data[i][0].split(',')) for i in range(len(txt_data))
]

In [29]:
txt_data[:3]

[('/nfs/turbo/coe-soto/dr/images_processed/31622_left.jpg',
  '/nfs/turbo/coe-soto/mmakar/multiple_shortcut/dr/experiment_data/rs0/noise_imgs/test_0.1/no_noise.png',
  '0.0',
  '0.0'),
 ('/nfs/turbo/coe-soto/dr/images_processed/1197_right.jpg',
  '/nfs/turbo/coe-soto/mmakar/multiple_shortcut/dr/experiment_data/rs0/noise_imgs/test_0.1/no_noise.png',
  '0.0',
  '0.0'),
 ('/nfs/turbo/coe-soto/dr/images_processed/29176_left.jpg',
  '/nfs/turbo/coe-soto/mmakar/multiple_shortcut/dr/experiment_data/rs0/noise_imgs/test_0.1/no_noise.png',
  '0.0',
  '0.0')]

In [30]:
test_data = pd.read_csv(
    f'{experiment_directory}/test_{dist}.txt'
).values.tolist()

test_data = [
    tuple(test_data[i][0].split(',')) for i in range(len(test_data))
]


In [31]:
test_data[:3]

[('/nfs/turbo/coe-soto/dr/images_processed/24297_right.jpg',
  '/nfs/turbo/coe-soto/mmakar/multiple_shortcut/dr/experiment_data/rs0/noise_imgs/test_0.1/no_noise.png',
  '0.0',
  '0.0'),
 ('/nfs/turbo/coe-soto/dr/images_processed/323_left.jpg',
  '/nfs/turbo/coe-soto/mmakar/multiple_shortcut/dr/experiment_data/rs0/noise_imgs/test_0.1/no_noise.png',
  '0.0',
  '0.0'),
 ('/nfs/turbo/coe-soto/dr/images_processed/32149_right.jpg',
  '/nfs/turbo/coe-soto/mmakar/multiple_shortcut/dr/experiment_data/rs0/noise_imgs/test_0.1/32149_right.png',
  '4.0',
  '1.0')]

In [10]:
test_data.group.value_counts(normalize=True)

0.0-0.0    0.635775
2.0-1.0    0.131896
0.0-1.0    0.073244
1.0-1.0    0.060218
3.0-1.0    0.025340
2.0-0.0    0.020713
4.0-1.0    0.020642
1.0-0.0    0.012670
3.0-0.0    0.009823
4.0-0.0    0.009680
Name: group, dtype: float64

In [21]:
test_data_sub = test_data.groupby('group').sample(frac=0.2, random_state=0).reset_index(drop=True)

In [22]:
test_data_sub.group.value_counts(normalize=True)

0.0-0.0    0.635587
2.0-1.0    0.132028
0.0-1.0    0.073310
1.0-1.0    0.060142
3.0-1.0    0.025267
2.0-0.0    0.020641
4.0-1.0    0.020641
1.0-0.0    0.012811
3.0-0.0    0.009964
4.0-0.0    0.009609
Name: group, dtype: float64

In [23]:
test_data_sub.head()

,img_name,noise_img,y0,y1,group
0,/nfs/turbo/coe-soto/dr/images_processed/31622_...,/nfs/turbo/coe-soto/mmakar/multiple_shortcut/d...,0.0,0.0,0.0-0.0
1,/nfs/turbo/coe-soto/dr/images_processed/1197_r...,/nfs/turbo/coe-soto/mmakar/multiple_shortcut/d...,0.0,0.0,0.0-0.0
2,/nfs/turbo/coe-soto/dr/images_processed/29176_...,/nfs/turbo/coe-soto/mmakar/multiple_shortcut/d...,0.0,0.0,0.0-0.0
3,/nfs/turbo/coe-soto/dr/images_processed/449_le...,/nfs/turbo/coe-soto/mmakar/multiple_shortcut/d...,0.0,0.0,0.0-0.0
4,/nfs/turbo/coe-soto/dr/images_processed/3640_r...,/nfs/turbo/coe-soto/mmakar/multiple_shortcut/d...,0.0,0.0,0.0-0.0


In [4]:
model_name = 'weighted_hsic'
xv_mode = 'two_step'
py1y0 = 0.9
curr_pred = pd.read_csv(
    (f'{base_dir}/final_models/all_valid_pred_{model_name}_{xv_mode}'
     f'_pix{pixel}_bs{batch_size}_py1y0{py1y0}_epochs2.csv')) 


In [48]:
pd.concat([curr_pred, pd.get_dummies(curr_pred.y0, prefix='y0')], axis=1).columns

Index(['y0', 'y1', 'predictions0', 'predictions1', 'predictions2',
       'predictions3', 'predictions4', 'pred_class', 'sample_weights', 'dist',
       'model', 'y0_0.0', 'y0_1.0', 'y0_2.0', 'y0_3.0', 'y0_4.0'],
      dtype='object')

In [47]:
curr_pred.shape

(1264400, 11)

In [26]:
curr_pred_stack = get_weighted_micro_auc(curr_pred)

<ipython-input-25-827666b01581>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_df['y_true'] = label_df.iloc[:, int(y_val)]
/home/mmakar/miniconda3/envs/slabs/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [27]:
curr_pred_stack.head()

,y_pred,sample_weights,y_true
0,0.921123,0.764149,1
1,0.948615,0.764149,1
2,0.935561,0.764149,1
3,0.944743,0.764149,1
4,0.940090,0.764149,1


In [28]:
curr_pred.shape

(1264400, 11)

In [29]:
curr_pred_stack.shape

(6322000, 3)

In [30]:
5 * 1264400

6322000

In [13]:
import pandas as pd
import numpy as np
experiment_directory = (f'/nfs/turbo/coe-soto/mmakar/multiple_shortcut/dr/experiment_data/'
    f'rs0')
data = pd.read_csv(
    f'{experiment_directory}/train.txt')
data = data['0'].str.split(",", expand=True)
data.columns = ['eye_img', 'noise_img', 'y0', 'y1']
for i in [0, 1]:
    data[f'y{i}'] = data[f'y{i}'].astype(np.float32)

In [14]:
data.y1[(data.y0==0.0)].value_counts(normalize=True)

1.0    0.89391
0.0    0.10609
Name: y1, dtype: float64

In [15]:
data.y1[(data.y0==1.0)].value_counts(normalize=True)

0.0    0.828397
1.0    0.171603
Name: y1, dtype: float64

In [16]:
data.y1[(data.y0==2.0)].value_counts(normalize=True)

0.0    0.85797
1.0    0.14203
Name: y1, dtype: float64